# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import DataScienceHelperLibrary as dsh
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
import numpy as np
import pandas as pd
import re
import seaborn as sns

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ben\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
database = 'MyDisasterResponse.db'
table = 'MyMessage'

engine = create_engine('sqlite:///{}'.format(database))

df = pd.read_sql_table(table, engine)

In [3]:
dsh.AnalyzeDataFrame(df)

-------------------- Dataframe analysis started --------------------
Shape:  (22621, 41)
Number of duplicate rows:  0
-------------------- Analysis of Columns with NaN values --------------------
Columns having all values: 40, 97.56%
index, type: int64                     0.0
id, type: int64                        0.0
message, type: object                  0.0
genre, type: object                    0.0
related, type: int64                   0.0
request, type: int64                   0.0
offer, type: int64                     0.0
aid_related, type: int64               0.0
medical_help, type: int64              0.0
medical_products, type: int64          0.0
search_and_rescue, type: int64         0.0
security, type: int64                  0.0
military, type: int64                  0.0
child_alone, type: int64               0.0
water, type: int64                     0.0
food, type: int64                      0.0
shelter, type: int64                   0.0
clothing, type: int64              

Hmm... there were nan values that have been inserted into database...

In [4]:
df[df.original == np.nan]

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


Hmmm.... there were definitely missing values...

In [5]:
type(df.at[7408, 'original']) # example ID I radonmly found.

NoneType

In [6]:
df[df.original == None]

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


In [7]:
type(None)

NoneType

Hmmm.... I thought that None or np.nan would be interpretet as something like DBNULL. Instead it is a string value?! I didn't calculated that...

In [8]:
df[df.original == 'NoneType']

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


I give up... I thought there is an adequate handling for np.nan und database NULL values. I just intended to check the number of mising values before and after writing into database. But as you can see, currently I'm out of ideas how to do it ... And I don't intend to waste time with that.

## Remark: Dropping rows before modeling

As I mentioned at ETL 7.0, I didn't drop rows that were irrelevant because its possible that irrelevant rows have been added before selecting the data from database. 

So here at this step the data needs to be cleaned either way. My following steps:
- remove column 'original' as there are many missing values as explored during ETL and cannot be proven here as you can see above ^^
- remove rows when message starts with 'NOTES:'
- remove rows with related not in [0, 1]
- remove rows that have no category at all
- remove rows with messages that are much longer than the length of all messages
- remove rows that would be empty after message has gone through Tokenize( ) so please allow me at this place to slightly change the order of to dos :)

In [9]:
df = dsh.RemoveColumnsByWildcard(df, 'original')

-------------------- Start finding and removing columns matchting to wildcards: ['original'] --------------------
Columns found to remove:  ['original']
-------------------- Finished removing columns matchting to wildcards --------------------


In [10]:
df[df['message'].str.startswith('NOTES:')]

,index,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
3062,3083,3539,NOTES: Personal. Not an emergency.,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3095,3116,3579,NOTES: aclet dry We are counting on you.,direct,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3099,3120,3583,NOTES: talking about gas prices,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3162,3183,3651,NOTES: Historic earthquake in Haiti.,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3236,3257,3737,NOTES: They are requesting that somebody to pa...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3258,3279,3761,NOTES: Regular gossip or message sharing. Not ...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3268,3289,3775,NOTES: personal request. Not an emergency.,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3311,3332,3820,NOTES: regular gossip. Not an emergency.,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3357,3378,3871,NOTES: Gossip. Not an emergency.,direct,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3362,3383,3877,NOTES: Personal comment. Not an emergency.,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Remove messages starting with 'NOTES:'

df = dsh.RemoveRowsWithValueInColumn(df, 'message', ['NOTES:'], option = 'startswith')

329 rows (ca. 1.45%) have been removed having value/s "['NOTES:']" in column "message"


In [12]:
# Just keep messages with related = 0 or 1 (drop 2 or potentially other values)

df = dsh.SelectRowsWithValueInColumn(df, 'related', [0, 1])

165 rows (ca. 0.73%) have been removed not having value/s "[0, 1]" in column "related"


In [13]:
# Remove rows that have no category at all

# Get all categories
colsToSum = list(dsh.RemoveColumnsByWildcard(df, ['index', 'id', 'message', 'original', 'genre']))

shapeBefore = df.shape
dfRemovedNoCategory = df[df[colsToSum].sum(axis = 1) == 0]
df = df[df[colsToSum].sum(axis = 1) > 0]

print('Rows removed that had no category: ', shapeBefore[0] - df.shape[0])

-------------------- Start finding and removing columns matchting to wildcards: ['index', 'id', 'message', 'original', 'genre'] --------------------
Columns found to remove:  ['index', 'id', 'message', 'genre']
-------------------- Finished removing columns matchting to wildcards --------------------
Rows removed that had no category:  5398


In [14]:
# Remove messages that are much longer than the average

df = dsh.RemoveRowsByValuesOverAverage(df, 'message', 4)

-------------------- Start dropping rows with value/textlength > 4 * mean --------------------
Rows removed:  88
-------------------- Finished removing --------------------


### 2. Write a tokenization function to process your text data

In [15]:
class TokenizerSettings:
    
    SupportedLanguages = [
    'english', 
    #'frensh',
    #'haitian'
    ]
    
    ConsiderOnlyAlphabet = 1
    ConsiderMinimumLength = 3
    
    CleanUrlReg = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    UrlPlaceHolder = ''    
    
    UseLemmatizer = 1
    UseStemmer = 1
    UseStopWords = 1
    UseTokenizer = 1
    UseWordNet = 1

    WordMinLengthForPortStemmer = 5
    
    # By hazard I saw a message with that organization/institution,
    # filtered roughtly 40 rows and decided to consider/give this word a weight
    # and not just drop it.
    # In my eyes this is not overfitting because new words need to be recognized that potentially have
    # influence. And I intended to make it parameterizable.
    WhiteListWords = set(['UNHCR']) 
    
def Tokenize(text):
    
    text = text.lower() # I prefer calling this function once instead for each word
    
    if not dsh.IsNullOrEmpty(TokenizerSettings.CleanUrlReg):
        foundUrls = re.findall(TokenizerSettings.CleanUrlReg, text)
        
        for url in foundUrls:
            text = text.replace(url, TokenizerSettings.UrlPlaceHolder)
    
    tokens = word_tokenize(text)

    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    # Call str() if values have been passed to whitelistwords that are no strings
    whiteList = [str(white).lower() for white in TokenizerSettings.WhiteListWords]
    cleanTokens = []
    for tok in tokens:

        if TokenizerSettings.ConsiderOnlyAlphabet:
            tok = dsh.RemoveNonLetters(tok)
            if dsh.IsNullOrEmpty(tok):
          #      print('Empty')
                continue
        
        if TokenizerSettings.UseLemmatizer:
            tok = lemmatizer.lemmatize(tok).strip()
        
        #if TokenizerSettings.UseStemmer and len(tok) >= TokenizerSettings.WordMinLengthForPortStemmer:
            #stemmed = stemmer.stem(tok)
        
        if len(tok) < TokenizerSettings.ConsiderMinimumLength:
            #print('Too short')
            continue
        
        isStopWord = False
        if TokenizerSettings.UseStopWords:
            try:
                for lang in list([lng for lng in TokenizerSettings.SupportedLanguages if lng is not None and len(lng) > 0]):
                    if tok in stopwords.words(lang):
                        isStopWord = True
                        break
            except:
                print('Error during stop word handling with language: ', lang, \
                    '\nand token: ', tok)
        if isStopWord:
            #print('Stopword')
            continue
        
        isUnknownWord = False
        if TokenizerSettings.UseWordNet:
            if not wordnet.synsets(tok):
                if not tok in whiteList:
                    isUnknownWord = True
        if isUnknownWord:
            continue
        
        if tok in cleanTokens:
            #print('Contained')
            continue
        
        cleanTokens.append(tok)
    
    return cleanTokens

In [ ]:
# Now remove those rows with empty message after going through Tokenize()

dsh.PrintLine('Removing potentially empty rows after Tokenize')
print('Please be paitent, this step may take some seconds')
df['dropRow'] = df['message'].apply(lambda x: 1 if len(Tokenize(x)) == 0 else 0)
dsh.PrintLine('Finished')

df[df['dropRow'] == 1]

Error during stop word handling with language:  english 
and token:  nothing
Error during stop word handling with language:  english 
and token:  note


So these 37 "rubbish" messages can smoothly be dropped.

In [17]:
df = dsh.RemoveRowsWithValueInColumn(df, 'dropRow', 1)

37 rows (ca. 0.22%) have been removed having value/s "[1]" in column "dropRow"


In [18]:
df = df.drop('dropRow', axis = 1)

In [19]:
list(df[df.index == 22611].message)

['UNHCR is also distributing tents, plastic sheets, sleeping mats and hygiene and sanitation items in unflooded areas of Leitchuor and Nip Nip.']

In [ ]:
df[df.message.str.contains('UNHCR')]

In [16]:
Tokenize('UNHCR is also distributing tents, plastic sheets, sleeping mats and hygiene and sanitation items in unflooded areas of Leitchuor and Nip Nip.')

['unhcr',
 'also',
 'distributing',
 'tent',
 'plastic',
 'sheet',
 'sleeping',
 'mat',
 'hygiene',
 'sanitation',
 'item',
 'area',
 'nip']

In [17]:
X = dsh.SelectColumnsByWildcard(df, ['message'])

Y = dsh.RemoveColumnsByWildcard(df, ['index', 'id', 'message', 'original', 'genre'])

-------------------- Start finding and keeping columns matchting to wildcards: ['message'] --------------------
-------------------- Finished keeping columns matchting to wildcards: 1 --------------------
-------------------- Start finding and removing columns matchting to wildcards: ['index', 'id', 'message', 'original', 'genre'] --------------------
Columns found to remove:  ['index', 'id', 'message', 'genre']
-------------------- Finished removing columns matchting to wildcards --------------------


### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [18]:
# pipeline = Pipeline([
#         ('features', FeatureUnion([

#             ('text_pipeline', Pipeline([
#                 ('vect', CountVectorizer(tokenizer = Tokenize)),
#                 ('tfidf', TfidfTransformer())
#             ])),

#             #('starting_verb', StartingVerbExtractor())
#         ])),

#         ('clf', MultiOutputClassifier())
#     ])


_cv = CountVectorizer(tokenizer = Tokenize)
_tfidf = TfidfTransformer()
_classifierInner = RandomForestClassifier(n_estimators = 7, random_state = 42)
_classifierOuter = MultiOutputClassifier(_classifierInner, n_jobs = 4)


pipeline = Pipeline([('vect', _cv),
                     ('tfidf', _tfidf),
                     ('clf', _classifierOuter,)
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

## Remark: Cleaning again
Before I split data, I remove those rows that would be null/none/empty after going through Tokenize() to avoid bad side effects. For example I don't konw how a training message is treated when it has no weights.

In [19]:
XTrain, XTest, yTrain, yTest = dsh.SplitDataTrainTest(X, Y)

-------------------- Test/Train split (test size = 0.3, random state = 42: --------------------
Training: X = (11879, 1), y = (11879, 36)
Test    : X = (5091, 1), y = (5091, 36)
--------------------  --------------------


In [20]:
xFitTrans = pipeline.fit(XTrain['message'].values, yTrain.values)

# Remark: It took some days until I realized that I passed the dataframe to fit instead the values of the message column -.-
#         I was really blind...

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [21]:
yPred = xFitTrans.predict(XTest['message'].values)

I just intended to verify for myself that both is the same.

In [32]:
from sklearn.metrics import classification_report

for ind in range(yPred.shape[1]):
    colName = yTest.columns[ind]
    dsh.PrintLine('Column: ' + str(colName))
    dsh.PrintLine('Prediction')
    print(classification_report(yTest[colName].values, yPred[:, ind]))
dsh.PrintLine()

-------------------- Column: related --------------------
-------------------- Prediction --------------------
             precision    recall  f1-score   support

          1       1.00      1.00      1.00      5091

avg / total       1.00      1.00      1.00      5091

-------------------- Column: request --------------------
-------------------- Prediction --------------------
             precision    recall  f1-score   support

          0       0.80      0.91      0.85      3898
          1       0.48      0.27      0.35      1193

avg / total       0.73      0.76      0.74      5091

-------------------- Column: offer --------------------
-------------------- Prediction --------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5067
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5091

-------------------- Column: aid_related --------------------
---------------

C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Python\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [33]:
ind = 0
colName = yTest.columns[ind]
classifyReport = classification_report(yTest[colName].values, yPred[:, ind])


In [34]:
print(yTest.shape, yPred.shape)

(5091, 36) (5091, 36)


In [35]:
cntErr = 0
dirError = {}
lstFalses = []
for cind in range(yTest.shape[1]):
    
    cntErr = 0
    colname = yTest.columns[cind]
    
    for ind in range(yTest.shape[0]):
        
        if yTest.iloc[ind, cind] != yPred[ind, cind]:
            cntErr += 1
            lstFalses.append(str(colname) + ' - ' + str(XTest.iloc[ind]['message']))
    
    err = cntErr * 100 / yTest.shape[0]
    dirError[str(colname)] = err
    
dfError = pd.DataFrame({'Errors' : list(dirError.values())}, index = yTest.columns)

In [ ]:
for val in lstFalses:
    print(Tokenize(val))

print('False predicted')

['request', 'need', 'tent', 'spray', 'mosquito']
['request', 'found', 'number', 'net', 'instruction', 'best', 'send', 'exact']
['request', 'much', 'frozen', 'food', 'left', 'wonder', 'sandy']
['request', 'international', 'hospital', 'staff', 'lived', 'tent', 'city', 'sweltered', 'temperature', 'degree', 'celsius']
['request', 'long', 'distribution', 'last', 'area', 'please', 'victim', 'disaster']
['request', 'west', 'climate', 'varies', 'subtropical', 'moderate']
['request', 'yes', 'need', 'lot', 'aid']
['request', 'writing', 'yesterday', 'asking', 'help', 'yet', 'receive', 'shelter', 'still', 'suffering']
['request', 'surveillance', 'official', 'found', 'four', 'known', 'dengue', 'viral', 'strain', 'island', 'two', 'three', 'observed', 'highly', 'active', 'epidemic', 'causing', 'haemorrhagic', 'fever', 'shock', 'syndrome']
['request', 'child', 'sick', 'take']
['request', 'found', 'cap', 'haitian', 'house', 'destroyed']
['request', 'true', 'president', 'said', 'build', 'house', 'haiti'

['request', 'sends', 'condolence', 'give', 'haitian', 'people', 'died', 'event', 'january', 'courage']
['request', 'people', 'participate', 'planning', 'rebuilding']
['request', 'people', 'tent', 'impasse', 'area', 'lot', 'problem', 'need', 'food', 'water']
['request', 'south', 'along', 'ethiopia', 'border', 'widespread', 'weekly', 'rainfall', 'total', 'exceeding', 'seen', 'local']
['request', 'shelter', 'continued', 'assistance', 'high', 'altitude', 'village', 'material', 'technical', 'support', 'create', 'warm', 'room', 'provision', 'repair', 'kit', 'winterised', 'tent', 'existing']
['request', 'men', 'yon', 'map', 'please', 'god', 'save', 'wall', 'child', 'thirst', 'hunger', 'going', 'kill', 'call', 'cut']
['request', 'want', 'remove', 'rubble']
['request', 'true', 'people', 'senegal', 'switzerland', 'visa']
['request', 'waiting', 'help', 'need', 'money', 'food', 'feed', 'child', 'sick', 'life', 'carrefour', 'midway', 'street', 'impasse', 'villa']
['request', 'growing']
['request', 

['request', 'please', 'need', 'tent', 'prelate', 'food', 'thank']
['request', 'includes', 'cassava', 'maize', 'equivalent', 'source', 'national', 'regional']
['request', 'house', 'help', 'way', 'signed', 'jean']
['request', 'sister', 'niece', 'nephew', 'courage', 'time', 'hang', 'give']
['request', 'wrote', 'never', 'brought', 'need', 'know', 'longer', 'help', 'thank', 'location']
['request', 'large', 'build', 'moraine', 'end', 'glacier', 'retains', 'water', 'ice', 'holding', 'together', 'melt']
['request', 'see', 'making', 'way', 'god', 'never', 'let', 'shame', 'people', 'really', 'trust']
['request', 'citizen', 'take', 'step', 'immigration', 'find', 'family', 'older']
['request', 'also', 'diver', 'searching', 'riverbed']
['request', 'nigga', 'went', 'hard', 'wrote', 'paragraph', 'exaggerating', 'hurricane', 'shit', 'storm', 'time']
['request', 'help', 'catch', 'lot', 'microbe', 'water', 'drinking', 'capital']
['request', 'come', 'back', 'nothing', 'clothes', 'put']
['request', 'live'

['request', 'kind', 'cultivated', 'interaction', 'people', 'event', 'escaping', 'reality']
['request', 'like', 'find', 'information', 'question', 'asked']
['request', 'increased', 'range', 'locally', 'sourced', 'foodstuff', 'help', 'improve', 'dietary', 'intake', 'community']
['request', 'case', 'earthquake']
['request', 'tell', 'civil', 'officer', 'get', 'converted', 'accept', 'jesus']
['request', 'however', 'worker', 'routinely', 'handle', 'corps', 'may', 'risk', 'contracting', 'tuberculosis', 'virus', 'hepatitis', 'hiv', 'gastrointestinal', 'infection', 'rotavirus', 'diarrhoea', 'salmonellosis', 'fever', 'shigellosis', 'cholera']
['request', 'alternative', 'safer', 'housing', 'material', 'report', 'titled', 'relief', 'violation', 'human', 'problem', 'heat', 'degree', 'temporary', 'shelter', 'called', 'cowshed', 'shoebox', 'well', 'known']
['request', 'like', 'know', 'earthquake', 'happen']
['request', 'way', 'see', 'country', 'situation', 'better', 'handing', 'back', 'white', 'one',

['request', 'need', 'tent']
['request', 'please', 'like', 'know', 'true', 'moment']
['request', 'key', 'party', 'house', 'walk', 'shame']
['request', 'director', 'general', 'pakistan', 'meteorological', 'department', 'told', 'percent', 'normal', 'rain', 'expected', 'winter', 'though', 'added', 'major', 'impact', 'agriculture', 'mainstay', 'economy']
['request', 'tablet', 'trait', 'water', 'give']
['request', 'see', 'never', 'write']
['request', 'house', 'crashed', 'earthquake']
['request', 'like', 'thank', 'country', 'responded', 'positively', 'actively', 'haiti', 'natural', 'disaster', 'minute']
['request', 'take', 'care', 'situation', 'observe', 'like', 'rest']
['request', 'still', 'received', 'help', 'please', 'come', 'need', 'thanks', 'advance']
['request', 'school', 'reopened']
['request', 'please', 'circulate', 'message', 'authority', 'send', 'food', 'victim', 'sought', 'refuge', 'mark']
['request', 'route', 'forgotten']
['request', 'house', 'cant', 'get', 'need', 'tent']
['reque

...

['earthquake', 'like', 'way', 'give', 'information', 'know', 'tsunami', 'safe', 'self']
['earthquake', 'want', 'donate', 'haiti', 'relief', 'http', 'pol']
['earthquake', 'starting', 'sandy', 'hurricane', 'named', 'woman', 'arrive', 'wet', 'wild', 'leave', 'house', 'official', 'resident', 'least', 'hour', 'stranded', 'rumor', 'storm', 'hitting', 'suddenly', 'grown', 'vagina']
['earthquake', 'provide', 'shelter', 'people', 'zone', 'near', 'hotel', 'flamboyant', 'right', 'side', 'ground', 'road', 'water', 'food']
['earthquake', 'hello', 'living', 'street', 'hear', 'pub', 'daily', 'wrote', 'receive', 'thing', 'want', 'know', 'try', 'foul', 'people']
['earthquake', 'note', 'message', 'work']
['earthquake', 'haiti', 'hit', 'largest', 'year', 'http']
['earthquake', 'group', 'coordination', 'interactive', 'asking', 'aid', 'people', 'zone', 'rear', 'neighbourhood', 'alone', 'difficult', 'moment']
['earthquake', 'additional', 'train', 'service', 'commenced', 'provide', 'transportation', 'strande

['cold', 'pakistan', 'need', 'volunteer']
['cold', 'plenty', 'clothes', 'also', 'bring', 'canned', 'item', 'make', 'food']
['cold', 'need', 'ensure', 'sufficient', 'supply', 'hand', 'distributed', 'equitably', 'ideally', 'utilising', 'local', 'distribution', 'process']
['cold', 'group', 'spanish', 'local', 'info', 'concepcion', 'please', 'chile']
['cold', 'tokyo', 'dec', 'japan', 'tuesday', 'sent', 'maritime', 'self', 'defense', 'force', 'convoy', 'including', 'helicopter', 'water', 'thailand', 'help', 'search', 'missing', 'people', 'following', 'sunday', 'earthquake', 'subsequent', 'tsunami', 'southeast', 'south', 'asian', 'region']
['cold', 'message', 'received', 'french', 'creole']
['cold', 'china', 'lie', 'right', 'region', 'eurasian', 'pacific', 'indian', 'ocean', 'plate', 'meet', 'suffers', 'frequent', 'earthquake', 'due', 'tectonic', 'movement']
['cold', 'like', 'know', 'minister', 'education', 'teacher', 'pay', 'third', 'cycle']
['cold', 'soil', 'erosion', 'still', 'continuing'

['flood', 'control', 'worker', 'preparing', 'highest', 'crest', 'year', 'yangtze', 'river', 'state', 'medium', 'said', 'monday']
['said', 'also', 'working', 'set', 'pakistani', 'institute', 'legislative', 'study', 'strengthen', 'capacity', 'member', 'staff', 'national', 'provincial', 'legislature']
['house', 'destroyed', 'back', 'inside', 'engineer', 'fir']
['currently', 'mobilizing', 'team', 'respond', 'catastrophic', 'earthquake', 'struck', 'haiti', 'today', 'make', 'impact']
['occasional', 'squall', 'rough', 'sea', 'experienced', 'along', 'myanmar', 'coast', 'surface', 'wind', 'speed', 'reaching', 'mph', 'according']
['chef', 'offer', 'service', 'food', 'bank', 'facility', 'provide', 'relief', 'victim', 'storm', 'sandy']
['know', 'country', 'lot', 'problem']
['availability', 'herbicide', 'pesticide', 'fungicide', 'problematic', 'throughout', 'country', 'uncommon', 'experience', 'farmer', 'purchase', 'adulterated', 'expired', 'chemical', 'market']
['said', 'department', 'also', 'look

['complex', 'consist', 'three', 'classroom', 'teacher', 'home', 'toilet', 'block']
['people', 'also', 'died', 'due', 'landslide', 'triggered', 'incessant', 'heavy', 'rain', 'various', 'part', 'state', 'past', 'two', 'week']
['also', 'destination', 'refugee', 'several', 'neighbor', 'witnessing', 'security', 'turbulence', 'estimated', 'many', 'actual', 'number', 'much', 'higher', 'let', 'alone', 'recurrent', 'food', 'crisis', 'seasonal', 'flood']
['family', 'member', 'chile', 'well', 'safe', 'zero', 'damage', 'home', 'santiago']
['january', 'islamabad', 'pakistan', 'direct', 'relief', 'extensive', 'experience', 'providing', 'assistance', 'aftermath', 'earthquake', 'shown', 'reconstructive', 'surgery', 'rehabilitation', 'prosthetics', 'assistive', 'device', 'emerge', 'significant', 'challenge', 'many', 'victim', 'traumatic', 'injury']
['regarding', 'crop', 'groundnut', 'production', 'expected', 'rise', 'throughout', 'province', 'copra', 'yield', 'similar', 'last', 'year']
['never', 'seen'

['food', 'balanced', 'basket', 'addition', 'rice', 'noodle', 'baby', 'child', 'needed']
['haiti', 'earthquake', 'twitter', 'picture', 'sweep', 'across', 'web', 'photo', 'http']
['new', 'emergency', 'operation', 'work', 'swedish', 'rescue', 'service', 'agency', 'establishment', 'staff', 'base', 'camp', 'management', 'facility', 'equipment']
['latest', 'news', 'haiti']
['chile', 'curfew', 'concepcion', 'city', 'get', 'riot', 'people', 'first', 'lotting', 'shop', 'burn']
['haiti', 'earthquake', 'hearing', 'http']
['india', 'metrological', 'department', 'issued', 'general', 'local', 'warning', 'impending', 'thunder', 'squall', 'northeastern', 'region']
['increased', 'number', 'death', 'cattle', 'goat', 'suspected', 'caused', 'consumption', 'grass', 'leading', 'deterioration', 'animal', 'digestive', 'system', 'poisoning', 'fluorosis', 'excessive', 'fluoride']
['whole', 'food', 'open', 'line', 'real', 'sandy', 'school', 'still', 'power', 'closed', 'thursday']
['least', 'billion', 'private', 

In [36]:
dfError.sort_values('Errors',ascending = False)

,Errors
aid_related,46.847378
weather_related,36.672559
direct_report,28.167354
request,23.826360
other_aid,20.428207
food,17.344333
earthquake,14.358672
shelter,14.005107
storm,13.022982
medical_help,12.374779


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
_classifierOuter.get_params()

In [39]:


parameters = {
       # 'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
       # 'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
       # 'features__text_pipeline__vect__max_features': (None, 5000, 10000),
       # 'features__text_pipeline__tfidf__use_idf': (True, False),
      #  'clf__estimators': [RandomForestClassifier(10)],
      #  'clf__min_samples_split': [2, 3, 4],
      #  'features__transformer_weights': (
      #      {'text_pipeline': 1, 'starting_verb': 0.5},
      #      {'text_pipeline': 0.5, 'starting_verb': 1},
      #      {'text_pipeline': 0.8, 'starting_verb': 1},
      #  )
    
      #"classifier__max_depth": [3, None],
      #        "classifier__max_features": [1, 3, 10],
      #        "classifier__min_samples_split": [1, 3, 10],
      #        "classifier__min_samples_leaf": [1, 3, 10],
      #        # "bootstrap": [True, False],
      #        "classifier__criterion": ["gini", "entropy"]

    #'clf__estimator': [ AdaBoostClassifier(), RandomForestClassifier() ],
    'clf__estimator__n_estimators' : [400, 600, 800],
    'clf__estimator__max_features' : [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
    #'clf__estimator__max_depth' : [],
    #'clf__estimator__min_samples_split' : [],
    #'clf__estimator__min_samples_leaf' : [],
    #'clf__estimator__bootstrap' : [True, False],
    'clf__estimator__n_jobs' : [4],
    'clf__estimator__random_state' : [42],
    }
#parameters = _classifierOuter.get_params()

cv = GridSearchCV(pipeline, param_grid = parameters)

In [ ]:
cv.fit(XTrain['message'].values, yTrain.values)

In [37]:
cv.

SyntaxError: invalid syntax (<ipython-input-37-a20b2f444762>, line 1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
from sklearn.externals import joblib
joblib.dump(grid.best_estimator_, 'filename.pkl')

#https://stackoverflow.com/questions/34143829/sklearn-how-to-save-a-model-created-from-a-pipeline-and-gridsearchcv-using-jobli
#If you want to dump your object into one file - use:  

joblib.dump(grid.best_estimator_, 'filename.pkl', compress = 1)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.